# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H. Linz  ->  H. Linz  |  ['H. Linz']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
E. Nasedkin  ->  E. Nasedkin  |  ['E. Nasedkin']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
J. Liu  ->  J. Liu  |  ['J. Liu']
Arxiv has 55 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2405.10427
extracting tarball to tmp_2405.10427...

 done.


H. Linz  ->  H. Linz  |  ['H. Linz']


Found 150 bibliographic references in tmp_2405.10427/G323.46-0.08.bbl.
syntax error in line 5: unbalanced braces
Retrieving document from  https://arxiv.org/e-print/2405.10595
extracting tarball to tmp_2405.10595... done.
Retrieving document from  https://arxiv.org/e-print/2405.10841


...

 done.


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']


/tmp/ipykernel_2061/1211882699.py:51: LatexWarning: 2405.10866 did not run properly
list index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2405.10890
extracting tarball to tmp_2405.10890...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.10427-b31b1b.svg)](https://arxiv.org/abs/2405.10427) | **The accretion burst of the massive young stellar object G323.46 -0.08**  |
|| V. Wolf, et al. -- incl., <mark>H. Linz</mark> |
|*Appeared on*| *2024-05-20*|
|*Comments*| **|
|**Abstract**|            Accretion bursts from low-mass young stellar objects (YSOs) are known for many decades. In recent years, the first accretion bursts of massive YSOs (MYSOs) were observed. These phases of intense protostellar growth are of particular importance for studying massive star formation. Bursts of MYSOs are accompanied by flares of Class II methanol masers (hereafter masers), caused by an increase in exciting mid-infrared (MIR) emission. The G323.46$-$0.08 (hereafter G323) event extends the small sample of known MYSO bursts. Maser observations of the MYSO G323 show evidence of a flare, which was presumed to be caused by an accretion burst. This should be verified with IR data. We used time-dependent radiative transfer (TDRT) to characterize the heating and cooling timescales for eruptive MYSOs and to infer the main burst parameters. The G323 accretion burst is confirmed. It reached its peak in late 2013/early 2014 with a Ks-band increase of 2.5mag. TDRT indicates that the duration of the thermal afterglow in the far-infrared (FIR) can exceed the burst duration by years. The latter was proved by SOFIA observations, which indicate a flux increase of $(14.2\pm4.6)$% at $70\, \rm \mu m$ and $(8.5\pm6.1)$% at $160\, \mu$m in 2022 (2 years after the burst end). A one-sided light echo emerged that was propagating into the interstellar medium. The G323 burst is probably the most energetic MYSO burst observed so far. Within $8.4 \rm \, yrs$, an energy of $(0.9\pm_{0.8}^{2.5}) \times 10^{47}\,\rm erg$ was released. The short timescale points to the accretion of a compact body, while the burst energy corresponds to an accumulated mass of at least $(7\pm_{6}^{20})\,M_{Jup}$ and possibly even more if the protostar is bloated. In this case, the accretion event might have triggered protostellar pulsations, which give rise to the observed maser periodicity.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.10595-b31b1b.svg)](https://arxiv.org/abs/2405.10595) | **The first low-mass eclipsing binary within the fully convective zone from TMTS**  |
|| C. Liu, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-05-20*|
|*Comments*| *12 pages, 8 figures, and 5 tables*|
|**Abstract**|            We present a comprehensive photometric and spectroscopic analysis of the short-period ($\sim$5.32 hours) and low-mass eclipsing binary TMTSJ0803 discovered by Tsinghua-Ma Huateng Telescope for Survey (TMTS). By fitting the light curves and radial velocity data with the Wilson--Devinney code, we find that the binary is composed of two late spotted active M dwarfs below the fully convective boundary. This is supported by the discovery of a significant Balmer emission lines in the LAMOST spectrum and prominent coronal X-ray emission. In comparison with the typical luminosity of rapidly rotating fully convective stars, the much brighter X-ray luminosity ($L_{X}/L_{\rm{bol}} = 0.0159 \pm 0.0059$) suggests the stellar magnetic activity of fully convective stars could be enhanced in such a close binary system. Given the metallicity of [M/H] = $-$ 0.35 dex as inferred from the LAMOST spectrum, we measure the masses and radii of both stars to be $M_{1} = 0.169 \pm 0.010~M_{\odot}$, $M_{2} = 0.162 \pm 0.016~M_{\odot}$, $R_{1} = 0.170 \pm 0.006~R_{\odot}$, and $R_{2} = 0.156 \pm 0.006~R_{\odot}$, respectively. Based on the luminosity ratio from the light curve modeling, the effective temperatures of two components are also estimated. In comparison with the stellar evolution models, the radii and effective temperatures of two components are all below the isochrones. The radius deflation might be mainly biased by a small radial velocity (RV) data or (and) a simple correction on RVs, while the discrepancy in effective temperature might be due to the enhanced magnetic activity in this binary.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.10890-b31b1b.svg)](https://arxiv.org/abs/2405.10890) | **A Versatile Framework for Analyzing Galaxy Image Data by Implanting Human-in-the-loop on a Large Vision Model**  |
|| M. Fu, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-05-20*|
|*Comments*| *26 pages, 10 figures, to be published on Chinese Physics C*|
|**Abstract**|            The exponential growth of astronomical datasets provides an unprecedented opportunity for humans to gain insight into the Universe. However, effectively analyzing this vast amount of data poses a significant challenge. Astronomers are turning to deep learning techniques to address this, but the methods are limited by their specific training sets, leading to considerable duplicate workloads too. Hence, as an example to present how to overcome the issue, we built a framework for general analysis of galaxy images, based on a large vision model (LVM) plus downstream tasks (DST), including galaxy morphological classification, image restoration, object detection, parameter extraction, and more. Considering the low signal-to-noise ratio of galaxy images and the imbalanced distribution of galaxy categories, we have incorporated a Human-in-the-loop (HITL) module into our large vision model, which leverages human knowledge to enhance the reliability and interpretability of processing galaxy images interactively. The proposed framework exhibits notable few-shot learning capabilities and versatile adaptability to all the abovementioned tasks on galaxy images in the DESI legacy imaging surveys. Expressly, for object detection, trained by 1000 data points, our DST upon the LVM achieves an accuracy of 96.7%, while ResNet50 plus Mask R-CNN gives an accuracy of 93.1%; for morphology classification, to obtain AUC ~0.9, LVM plus DST and HITL only requests 1/50 training sets compared to ResNet18. Expectedly, multimodal data can be integrated similarly, which opens up possibilities for conducting joint analyses with datasets spanning diverse domains in the era of multi-message astronomy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.10841-b31b1b.svg)](https://arxiv.org/abs/2405.10841) | **The ESO SupJup Survey I: Chemical and isotopic characterisation of the late L-dwarf DENIS J0255-4700 with CRIRES$^+$**  |
|| S. d. Regt, et al. -- incl., <mark>P. Mollière</mark>, <mark>E. Nasedkin</mark> |
|*Appeared on*| *2024-05-20*|
|*Comments*| *Submitted to A&A*|
|**Abstract**|            It has been proposed that the distinct formation and evolution of exoplanets and brown dwarfs may affect the chemical and isotopic content of their atmospheres. Recent work has indeed shown differences in the $^{12}$C/$^{13}$C isotope ratio, provisionally attributed to the top-down formation of brown dwarfs and the core accretion pathway of super-Jupiters. The ESO SupJup Survey aims to disentangle the formation pathways of isolated brown dwarfs and planetary-mass companions using chemical and isotopic tracers. The survey uses high-resolution spectroscopy with the recently upgraded VLT/CRIRES$^+$ spectrograph, covering a total of 49 targets. Here, we present the first results: an atmospheric characterisation of DENIS J0255-4700, an isolated brown dwarf near the L-T transition. We analyse its K-band spectrum using a retrieval framework where the radiative transfer code petitRADTRANS is coupled to PyMultiNest. Gaussian Processes are employed to model inter-pixel correlations and we adopt an updated parameterisation of the PT-profile. Abundances of CO, H$_2$O, CH$_4$, and NH$_3$ are retrieved for this fast-rotating L-dwarf. The ExoMol H$_2$O line list provides a significantly better fit than that of HITEMP. A free-chemistry retrieval is strongly favoured over equilibrium chemistry, caused by an under-abundance of CH$_4$. The free-chemistry retrieval constrains a super-solar C/O-ratio of $\sim0.68$ and a solar metallicity. We find tentative evidence ($\sim3\sigma$) for the presence of $^{13}$CO, with a constraint on the isotope ratio of $\mathrm{^{12}C/^{13}C}=184^{+61}_{-40}$ and a lower limit of $\gtrsim97$, suggesting a depletion of $^{13}$C compared to the interstellar medium ($\sim68$). High-resolution, high signal-to-noise K-band spectra provide an excellent means to constrain the chemistry and isotopic content of sub-stellar objects, as is the main objective of the ESO SupJup Survey.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.10866-b31b1b.svg)](https://arxiv.org/abs/2405.10866) | **Galaxy And Mass Assembly (GAMA): Stellar-to-Dynamical Mass Relation II. Peculiar Velocities**  |
|| M. B. Dogruel, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-05-20*|
|*Comments*| *Accepted: 15th May 2024*|
|**Abstract**|            Empirical correlations connecting starlight to galaxy dynamics (e.g., the fundamental plane (FP) of elliptical/quiescent galaxies and the Tully--Fisher relation of spiral/star-forming galaxies) provide cosmology-independent distance estimation and are central to local Universe cosmology. In this work, we introduce the mass hyperplane (MH), which is the stellar-to-dynamical mass relation $(M_\star/M_\mathrm{dyn})$ recast as a linear distance indicator. Building on recent FP studies, we show that both star-forming and quiescent galaxies follow the same empirical MH, then use this to measure the peculiar velocities (PVs) for a sample of 2496 galaxies at $z<0.12$ from GAMA. The limiting precision of MH-derived distance/PV estimates is set by the intrinsic scatter in size, which we find to be $\approx$0.1~dex for both quiescent and star-forming galaxies (when modeled independently) and $\approx$0.11~dex when all galaxies are modeled together; showing that the MH is as good as the FP. To empirically validate our framework and distance/PV estimates, we compare the inferred distances to groups as derived using either quiescent or star-forming galaxies. A good agreement is obtained with no discernible bias or offset, having a scatter of $\approx$0.05~dex $\approx$12\% in distance. Further, we compare our PV measurements for the quiescent galaxies to the previous PV measurements of the galaxies in common between GAMA and the Sloan Digital Sky Survey (SDSS), which shows similarly good agreement. Finally, we provide comparisons of PV measurements made with the FP and the MH, then discuss possible improvements in the context of upcoming surveys such as the 4MOST Hemisphere Survey (4HS).         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2405.10427.md
    + _build/html/tmp_2405.10427/./Ksequence10_defroi.png
    + _build/html/tmp_2405.10427/./lc_all.png
    + _build/html/tmp_2405.10427/./naco_Z_slit_ruler.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mypm}{\mathbin{\smash{$
$\raisebox{0.65ex}{$
$            \underset{\raisebox{0.05ex}{\smash -}}{\smash+}$
$            }$
$        }$
$    }$
$}$
$\newcommand{\goodchi}{\protect\raisebox{2pt}{\chi}}$
$\newcommand{\ts}{ }$
$\newcommand{\arraystretch}{1.05}$
$\newcommand{\arraystretch}{1}$</div>



<div id="title">

# The accretion burst of the massive young stellar object G323.46$-$0.08$\thanks{Based on observations made with the NASA/DLR Stratospheric Observatory for Infrared Astronomy (SOFIA) under Proposal ID 75\_0103 and at the European Organization for Astronomical Research in the Southern Hemisphere under ESO programs ID 077.C-0687(A), 083.C-0582(A), 290.C-5165(A).}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2405.10427-b31b1b.svg)](https://arxiv.org/abs/2405.10427)<mark>Appeared on: 2024-05-20</mark> - 

</div>
<div id="authors">

V. Wolf, et al. -- incl., <mark>H. Linz</mark>

</div>
<div id="abstract">

**Abstract:** ${Accretion bursts from low-mass young stellar objects (YSOs) are knownformany decades. In recent years, the first accretion bursts of massive YSOs (MYSOs) were observed.These phases of intense protostellar growth are of particular importance for studying massive star formation. Bursts of MYSOs are accompanied by flares of Class II methanol masers (hereafter masers), caused by an increase in exciting mid-infrared (MIR) emission.They can lead to long-lasting thermal afterglows of the dust continuum radiation visible at infrared (IR) and (sub)mm wavelengths. Furthermore, they might cause a scattered light echo.The G323.46$-$0.08 (hereafter G323) event, which shows all these features, extendsthe small sample of known MYSO bursts.   }$ ${Maser observations of the MYSO G323show evidence of a flare, which was presumed to be caused by an accretion burst. This should be verified with IR data.We used time-dependent radiative transfer (TDRT) to characterize the heating and cooling timescales for eruptive MYSOs and to infer the main burst parameters.   }$ ${Burst light curves, as well as the pre-burst spectral energy distribution (SED) wereestablished from archival IR data.The properties of the MYSO, including its circumstellar disk and envelope, were derived by using static radiative transfer modeling of pre-burst data. For the first time, TDRT was used to predict the temporal evolution of the SED. Observations with SOFIA/HAWC+ were performedto constrain the burst energy from the strength of the thermal afterglow. Image subtraction and ratioing was applied to reveal the light echo.   }$ ${The G323 accretion burst is confirmed. It reached its peak in late 2013/early 2014 with a $K_{\rm s}$-band increase of ${\sim}${\ts}2.5{\ts}mag.Both $K_{\rm s}$-band and integrated maser flux densities follow an exponential decay.TDRT indicates that the duration of the thermal afterglow in the far-infrared (FIR) can exceed the burst duration by years.The latter was proved by SOFIA observations, which indicate a flux increase of $(14.2\pm4.6)$\% at $70  \rm \mu m$ and $(8.5\pm6.1)$\% at $160{\ts}\mu$m in 2022 (2 years after the burst end).A one-sided light echo emerged that was propagating into the interstellar medium.}$ ${The burst origin of the G323 maser flare is verified.TDRT simulationsrevealed the strong influence of the burst energetics and the localdust distribution on the strength and duration of the afterglow.The G323 burst is probably the most energetic MYSO burst observed so far. Within $8.4 \rm   yrs$, an energy of $(0.9\mypm_{0.8}^{2.5}) \times 10^{47} \rm erg$ was released. The short timescale points tothe accretion of a compact body, while the burst energy corresponds to an accumulated mass of at least$(7\mypm_{6}^{20}) M_{Jup}$and possibly even more if the protostar is bloated.In this case, the accretion event might have triggered protostellar pulsations, which give rise to the observed maser periodicity.The associated IR light echo is the second observed from a MYSO burst.}$

</div>

<div id="div_fig1">

<img src="tmp_2405.10427/./Ksequence10_defroi.png" alt="Fig20" width="100%"/>

**Figure 20. -** The $K_{\rm s}$-band difference images show the temporal evolution of the burst and the associated LEs, displayed using a log-scale. The prompt echo, which originates from the cloud core, is monopolar and spreads to the south-east at an PA of $\sim$ 135◦. A remote LE that appeared later traces denser structures of the ISM. The red polygon encloses the area from which its light curves were established. The FoV is $135$\arcsec$\times80$\arcsec$.$ (*fig:seq*)

</div>
<div id="div_fig2">

<img src="tmp_2405.10427/./lc_all.png" alt="Fig18" width="100%"/>

**Figure 18. -** Left:
	Light curves based on VVV(X) (black) and (NEO)WISE photometry (W1 - blue, $W2$ - red) as well as 6.7{$\ts$}GHz total maser flux (green, \citealp{greenExcitedstateHydroxylMaser2015, MacLeod:2021}). Vertical
 red and blue lines mark the dates of the
 the burst onset and first flare evidence
	from the
	6.035{$\ts$}GHz exOH maser
 \citep{MacLeod:2021}.
    The $K_{\rm s}$ rise was approximated by
    a polynomial,
    while its decay is roughly linear on a log-scale (dashed line).
    The (NEO)WISE magnitudes are shifted
    to match those of $K_{\rm s}$. The
    integrated maser flux is shown on a log-scale (right ordinate).
    Its scatter is due to the short-term periodicity.
    Right: $K_{\rm s}$(black), $i$(blue), and $z$(green) light curves, with $i$ and $z$ magnitudes shifted
    to match those of $K_{\rm s}$.
    The $z$ pre- and post-magnitudes agree within the errors.
 (*fig:lc*)

</div>
<div id="div_fig3">

<img src="tmp_2405.10427/./naco_Z_slit_ruler.png" alt="Fig6" width="100%"/>

**Figure 6. -** The NACO AO $K_{\rm s}$-band image (epoch 2009) using a log-stretch with contours of the 19{$\ts$}GHz radio continuum (white, \citealp{murphy:2010}) and from the $Z$-band pre-burst image (red, epoch 2010).
    Maser spots are marked by crosses in blue (\citealp{Caswell:2001}, epoch 1994) and red (\citealp{greenExcitedstateHydroxylMaser2015}, epoch 2011), with sizes indicating the position error. The black diamond is at the peak of the 1.4{$\ts$}mm emission while the black square marks the _ GAIA_ source. The yellow rectangle shows orientation and width of the ISAAC slit.
 (*fig:naco*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2405.10427"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

72  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
